In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter

from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist
from art.defences.preprocessor import LabelSmoothing


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=4, out_channels=10, kernel_size=5, stride=1)
        self.fc_1 = nn.Linear(in_features=4 * 4 * 10, out_features=100)
        self.fc_2 = nn.Linear(in_features=100, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 10)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x


# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

# Step 2: Apply Label Smoothing as a preprocessing defense
label_smoothing = LabelSmoothing(max_value=1.0, apply_fit=True, apply_predict=False)
_, y_train = label_smoothing(x_train, y_train)

# Step 2: Create the model

model = Net()

# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 3: Create the ART classifier

classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

# Step 4: Setup TensorBoard writer
writer = SummaryWriter(log_dir="./runs/mnist_training")

# Step 5: Train the ART classifier and log metrics to TensorBoard
for epoch in range(5):  # number of epochs
    classifier.fit(x_train, y_train, batch_size=64, nb_epochs=1)

    # Log training accuracy and loss to TensorBoard
    predictions_train = classifier.predict(x_train[:100])  # Predict on a small batch for monitoring
    train_accuracy = np.sum(np.argmax(predictions_train, axis=1) == np.argmax(y_train[:100], axis=1)) / len(y_train[:100])

    writer.add_scalar("Accuracy/train", train_accuracy, epoch + 1)
    writer.add_scalar("Loss/train", criterion(model(torch.tensor(x_train[:100]).to(torch.float32)),
                                               torch.tensor(np.argmax(y_train[:100], axis=1))).item(), epoch + 1)

# Step 4a: Save the model after training
torch.save(model.state_dict(), "mnist_model-improved.pth")

# Step 6: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

# Log test accuracy to TensorBoard
writer.add_scalar("Accuracy/test", accuracy, 1)

# Close the TensorBoard writer
writer.close()

Epoch 1/5, Loss: 0.21337406506547446
Epoch 2/5, Loss: 0.0959126464166204
Epoch 3/5, Loss: 0.08918782818202636
Epoch 4/5, Loss: 0.08274443624322729
Epoch 5/5, Loss: 0.07755123047407708
Accuracy on benign test examples: 97.72%
